In [43]:
# !pip install scrapy
# !pip install seleniumnn 
# !pip install beautifulsoup4 

In [44]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from scrapy.selector import Selector
import re
import time
import csv
from itertools import islice
import pandas as pd
import os

In [45]:
driver = webdriver.Chrome('./chromedriver')

In [46]:
driver.get(f'https://www.6pm.com/marty/c/homepage-new')

In [47]:
def get_cookie_jar(driver):
    cookies = driver.get_cookies()
    cookie_jar = {x['name']:x['value'] for x in cookies}
    
    return cookie_jar

cookie_jar = get_cookie_jar(driver)

In [48]:
# Pass cookies back to request page information
page = requests.get('https://www.6pm.com/marty/c/homepage-new', 
                    cookies = cookie_jar)

In [49]:
soup = BeautifulSoup(page.text, 'html.parser')

In [50]:
boot_page = soup.findAll('a',{'class':'gae-click*Main-Nav*Shoes*Women-s-Boots'})

In [51]:
boot_page

[<a class="gae-click*Main-Nav*Shoes*Women-s-Boots" href="/women-boots/CK_XARCz1wHAAQHiAgMYAQI.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/">Boots</a>]

In [52]:
boot_page_str = str(boot_page)
start = boot_page_str.find('href') + 6
end = boot_page_str.find('Boots</a>') - 2
boot_page_str[start:end]

'/women-boots/CK_XARCz1wHAAQHiAgMYAQI.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/'

In [53]:
URL = "http://6pm.com" + boot_page_str[start:end]
page1 = requests.get(URL, cookies = cookie_jar)
page1

<Response [200]>

In [54]:
driver.get(URL)

In [55]:
driver.current_url

'https://www.6pm.com/women-boots/CK_XARCz1wHAAQHiAgMBAhg.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/'

In [57]:
URL2 = "https://www.6pm.com/women-boots/CK_XARCz1wF6AtQEggECnBjAAQHiAgQBAhgP.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/"

In [58]:
URL2

'https://www.6pm.com/women-boots/CK_XARCz1wF6AtQEggECnBjAAQHiAgQBAhgP.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/'

In [59]:
def extract_links_from_soup(current_soup):
    links = []
    for link in current_soup.find_all('a', {'itemprop':'url'}):
        links.append(link.get('href'))  
    return links

In [60]:
def fill_images_to_dataframe(links, pd_images):
    for link in links:
        URL_current = "http://www.6pm.com" + link
#        print("current_search_page: ", URL_current)
        page_current = requests.get(URL_current, cookies = cookie_jar)
        soup_current = BeautifulSoup(page_current.text, 'html.parser')
#        print(soup_current.text)
        try: 
            image_link = soup_current.find('img',{'alt':'BOTT'})['src']
        except TypeError:
            print("This item doesn't have a sole picture!")
        else:
            image_name = URL_current.split("/")[4] + ".jpg"
#            print(image_name)
#            print(image_link)
            pd_shoe_current = pd.Series([image_link, image_name, link], index=['image_link', 'jpeg', 'shoes_link'])
            pd_images = pd_images.append(pd_shoe_current, ignore_index=True)
    return pd_images

In [61]:
def save_jpegs(pd_images):
    for index, row in pd_images.iterrows():
        with open(os.path.join('./04_Data/Shoe_sole_images_3/',row['jpeg']), "wb") as file1:
            image = requests.get(row['image_link'])
            file1.write(image.content)
            file1.close()

In [71]:
current_URL = URL2
signal = 0
pd_images = pd.DataFrame(columns=['image_link', 'jpeg', 'shoes_link'])
while(signal == 0):
    current_page = requests.get(current_URL, cookies = cookie_jar)
    print("current URL: ", current_URL)
    current_soup = BeautifulSoup(current_page.text, 'html.parser')
    links = extract_links_from_soup(current_soup)
    pd_images = fill_images_to_dataframe(links, pd_images, )
    save_jpegs(pd_images)
    time.sleep(3)
    try: 
        current_URL = "https://www.6pm.com" + current_soup.findAll('a', {'rel': 'next'})[0]['href']
    except:
        print("Key Error: End of Search!")
        signal = 1
        
    

current URL:  https://www.6pm.com/women-boots/CK_XARCz1wF6AtQEggECnBjAAQHiAgQBAhgP.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/
current URL:  https://www.6pm.com/women-boots/CK_XARCz1wF6AtQEggECnBjAAQHiAgQBAhgP.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=1
current URL:  https://www.6pm.com/women-boots/CK_XARCz1wF6AtQEggECnBjAAQHiAgQBAhgP.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=2
current URL:  https://www.6pm.com/women-boots/CK_XARCz1wF6AtQEggECnBjAAQHiAgQBAhgP.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=3
current URL:  https://www.6pm.com/women-boots/CK_XARCz1wF6AtQEggECnBjAAQHiAgQBAhgP.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=4
This item doesn't have a sole picture!
current URL:  https://www.6pm.com/women-boots/CK_XARCz1wF6AtQEggECnBjAAQHiAgQBAhgP.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=5
This item doesn't have a sole picture!
current URL:  https://www.6pm.com/women-boots/CK_XARCz1wF6AtQEggECnB

In [72]:
pd_images.shape

(3347, 3)

In [73]:
pd_images.head()

,image_link,jpeg,shoes_link
0,https://m.media-amazon.com/images/I/615eisAo-C...,carlos-by-carlos-santana-ashbury-black.jpg,/p/carlos-by-carlos-santana-ashbury-black/prod...
1,https://m.media-amazon.com/images/I/712gDBAdQq...,carlos-by-carlos-santana-ashbury-inkwell.jpg,/p/carlos-by-carlos-santana-ashbury-inkwell/pr...
2,https://m.media-amazon.com/images/I/61cn+6hWaM...,cl-by-laundry-valto-black-super-suede.jpg,/p/cl-by-laundry-valto-black-super-suede/produ...
3,https://m.media-amazon.com/images/I/71POFRQyv+...,cl-by-laundry-valto-whiskey-super-suede.jpg,/p/cl-by-laundry-valto-whiskey-super-suede/pro...
4,https://m.media-amazon.com/images/I/61czfS-u2B...,frye-lila-slouch-saddle-polished-soft-full-gra...,/p/frye-lila-slouch-saddle-polished-soft-full-...


In [81]:
pd_images.tail()

,image_link,jpeg,shoes_link,label
3342,https://m.media-amazon.com/images/I/71pbQa8Z07...,frye-bruce-pull-on-black.jpg,/p/frye-bruce-pull-on-black/product/9173300/co...,3342
3343,https://m.media-amazon.com/images/I/71ckqrxHKK...,frye-paige-tall-riding-black-calf-leather.jpg,/p/frye-paige-tall-riding-black-calf-leather/p...,3343
3344,https://m.media-amazon.com/images/I/71oSvcM67+...,frye-dorado-riding-black-leather.jpg,/p/frye-dorado-riding-black-leather/product/89...,3344
3345,https://m.media-amazon.com/images/I/71x9ZLZL9A...,frye-dorado-riding-dark-brown-leather.jpg,/p/frye-dorado-riding-dark-brown-leather/produ...,3345
3346,https://m.media-amazon.com/images/I/71RlOQLhwg...,frye-veronica-slouch-black-tumbled-full-grain.jpg,/p/frye-veronica-slouch-black-tumbled-full-gra...,3346


In [82]:
pd_images.to_csv("./04_Data/shoe_pd_images1.csv")